# Знакомство с `pandas`

>Полная документация [`pandas` documentation](http://pandas.pydata.org/pandas-docs/stable/).


##  DataFrame: импорт данных и подсчет статистик

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt



`read_csv()` в `pandas` позволяет нам легко импортитировать даныые. По умаолчавнию предполагаем, что данные разделены запятиыми. При этом можно указать, если нужно использовать другой.

In [ ]:
unemployment = pd.read_csv('data/country_total.csv')

Так мы создали `pandas` `DataFrame`. Можем посмотреть на присер данных с помощью метода `.head()`. По умолчанию этот иетод показывает нам заголовки и первые пять строк.

In [ ]:
unemployment.head()

Чтобы узнать количество строк в данных можем воспользоваться функцией `len()`. Но лучше воспользоваться аргументом `shape`.

In [ ]:
unemployment.shape

Еще один полезный метод `.describe()`, он позволяет нам увидеть общие статистики по данным

In [ ]:
unemployment.describe()

Можно заметить, что количество количество в строке `count` отличается, это означает, что у нас есть строки с пропущенными значениями.

### Импортировать данные по  ссылке

Выше, мы импортировали данные с помощью функции `read_csv`, указав путь к файлу. Но эта функция также позволяет нам считтать данные по ссылке.

Используя  `read_csv`, считаем данные по странам и запишем их в переменную `countries`.

In [ ]:
countries_url = 'https://raw.githubusercontent.com/pyaternev57/datasets/main/countries.csv'
countries = pd.read_csv(countries_url)

### метод `tail` 

Метод похожий на `head`, но показывающий последние строки датасета

In [ ]:
# используем метод tail
countries.tail(10)

In [ ]:
# используем метод describe
countries.describe()

## Смена имен колонок, индексирование и срезы 

Вернемся к первому датасету, одна из колонок называется `month`, но содержит также и информация про год

In [ ]:
unemployment.rename(columns={'month' : 'year_month'}, inplace=True)

Метод `.rename()` позволяет изменять названия колонок или строк в датасете. Как можно увидеть мы передаем `dict` как значение аргумента  `columns`. Мы такде поставили аргумент `inplace` равным `True`, что изменяет `DataFrame`, а не копирует его.

In [ ]:
unemployment['year_month'].head()

In [ ]:
unemployment['month']

In [ ]:
unemployment[['year_month', 'country']].head()

In [ ]:
unemployment.year_month.head()

In [ ]:
unemployment[:5]

`.loc` метод рабоатет с индексами. В качестве арнкмента он принимает имя колонки/индекс, список колонок/индексов или же срез колонок/индексов(`'a' : 'f'`).

Создадим `DataFrame`чтобы посмотреть как это рабоатет.

In [4]:
bacteria = pd.DataFrame({'bacteria_counts' : [632, 1638, 569, 115],
                         'other_feature' : [438, 833, 234, 298]},
                         index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

In [5]:
bacteria

,bacteria_counts,other_feature
Firmicutes,632,438
Proteobacteria,1638,833
Actinobacteria,569,234
Bacteroidetes,115,298


In [ ]:
bacteria.loc['Actinobacteria']

In [ ]:
bacteria.iloc[2:3]

In [ ]:
# выведите строки 1, 5, 6, 9
unemployment.iloc[[1, 5, 6, 9]]

In [ ]:
# выведите каждую пятую строку с 25 по 50 не включительно
unemployment.iloc[25:50:5]

Разделим колонку `year_month` в две отдельные колонки. Для этого сначала сменим тип данных с `float` на `integer`

In [ ]:
unemployment['year'] = unemployment['year_month'].astype(int)

In [ ]:
unemployment['month'] = ((unemployment['year_month'] - unemployment['year']) * 100).round(0).astype(int)

In [ ]:
unemployment.head(12)

Также мы легко можем менять колонки местами

In [ ]:
unemployment = unemployment[['country', 'seasonality',
                             'year_month', 'year', 'month',
                             'unemployment', 'unemployment_rate']]

In [ ]:
unemployment.head(10)

## Слияние данных

In [ ]:
countries.tail(3)

In [ ]:
country_names = countries[['country', 'country_group', 'name_en']].copy()

In [ ]:
country_names.head(2)

В `pandas` есть специальная функция для слияния.

In [ ]:
unemployment = pd.merge(unemployment, country_names, on='country')

Слияние обычно более сложное, чем в нашем примере. Если вы хотите совмещать данные по нескольким параметрам, надо передать список, как указано ниже

```
pd.merge(first, second, on=['name', 'id'])
```


Иногда нужно выполнять слияние по колонкам с разными именами. Чтобы сделать это воспользуемся аргументами `left_on` и `right_on` 

```
pd.merge(one, two, left_on='city', right_on='city_name')
```


In [ ]:
country_codes = country_names.rename({"country":"c_code"}, axis=1).drop("country_group", axis=1)
country_codes.head()

Используем `merge` для слияния `unemployment` и `country_codes` по кодам стран.

In [ ]:
unemployment_merged = pd.merge(unemployment, country_codes, left_on='country', right_on='c_code')
unemployment_merged.head()

## Уникальные и пропущенные значения

In [ ]:
unemployment.head()

In [ ]:
unemployment.name_en.unique()

Чтобы получить **количество уникальных стран**, можем воспользоваться методом `len()` или можем использвоать метод  `Series.nunique()`.

In [ ]:
unemployment.name_en.nunique()

Более интересно как много каждое значение встречается в наших данных. Для этого используем метод `.value_counts()`

In [ ]:
unemployment['name_en'].value_counts()

In [ ]:
unemployment['name_en'].value_counts().sort_index()

In [ ]:
unemployment['year'].min(), unemployment['year'].max()

Найдем как много значений пропущено в колонке  `unemployment_rate`

In [ ]:
unemployment['unemployment_rate'].isnull().sum()

Исключить пропущенные значения можно с помощью метода `dropna`

In [ ]:
unemployment.dropna().head()

In [ ]:
unemployment.dropna().shape

## GroupBy


Что если мы хотим узнать как много пропущенных значений на уровне стран? 

In [ ]:
unemployment['unemployment_rate_null'] = unemployment['unemployment_rate'].isnull()

Чтобы посчитать **количество пропущенных значений для каждой страны**, воспользуемся методом `.groupby()`.

In [ ]:
unemployment.groupby('name_en')['unemployment_rate_null'].sum()

In [ ]:
unemployment_rate = unemployment.groupby('name_en', as_index=False)['unemployment_rate_null'].sum()

In [ ]:
unemployment_rate.head(3)

In [ ]:
unemployment_rate['n_obs'] = unemployment.groupby('name_en')['name_en'].count().values

In [ ]:
unemployment_rate['null_percentage'] = unemployment_rate['unemployment_rate_null'] / unemployment_rate['n_obs']

In [ ]:
unemployment_rate

### попробуйте GroupBy 
 
Давайте найдем среднее значение unemployment rate для стран входящих в Евросоюз и сравним с остальными

1. используйте `groupby` по параметру "country_group"
2. выберите колонку "unemployment_rate" 
3. используйте метод `.mean()`

In [ ]:
unemployment.groupby('country_group')['unemployment_rate'].mean()

## Сохранение DataFrame в csv 

In [ ]:
unemployment_rate.to_csv('data/unemployment_missing.csv')

In [ ]:
! head -5 data/unemployment_missing.csv

In [ ]:
unemployment_rate.to_csv('data/unemployment_missing.csv', index=False, sep=';')

In [ ]:
!head -5 data/unemployment_missing.csv

Удалять строки/стоблцы можно с помощью метода `drop`

In [ ]:
unemployment.drop('unemployment_rate_null', axis=1, inplace=True)

In [ ]:
unemployment.head()

## Сортировка значений

In [ ]:
unemployment.sort_values('unemployment_rate', ascending=False).head()

Воспользовавшись этим методом мы создали копию `DataFrame`, отсортированной в порядке убывания и напечатали пераве 5 строк

In [ ]:
unemployment['seasonality'].unique()

In [ ]:
unemployment[unemployment['seasonality'] == 'sa'].sort_values('unemployment_rate', ascending=False)[:5]